In [1]:
# libs
import pandas as pd
import numpy as np

# read in data 
# grab every file from this folder C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\Synergistic_Computing_hf\benchmarking\new\output
# and put it into a dataframe

# read in every file name from the folder
import os
path = r'C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\Synergistic_Computing_hf\benchmarking\new\output'
files = os.listdir(path)

# create a dataframe for each file. use the file name as the dataframe name
for file in files:
    # read in the file
    temp_df = pd.read_csv(path + '\\' + file, sep=',')
    # remove the .txt from the file name
    df_name = file[:-4]
    # create a dataframe with the name of the file
    exec(df_name + " = temp_df")

In [2]:
# append all batch1 dataframes together. they names of all batch 1 dataframes start with batch1

# create a list of all batch1 dataframes. but ignore batch10
batch1_dfs = []
for file in files:
    if file.startswith('batch1') and not file.startswith('batch10'):
        batch1_dfs.append(file[:-4])

# create a dataframe with all batch1 dataframes appended together
batch1_df = pd.DataFrame()
for df in batch1_dfs:
    temp_df = eval(df)
    batch1_df = batch1_df.append(temp_df, ignore_index=True)

# order the rows by the 'PK' column
batch1_df = batch1_df.sort_values(by=['PK'])

# drop the column 'PK' and add a column named 'batch_number' with the value of 1
batch1_df = batch1_df.drop(columns=['PK'])
batch1_df['batch_number'] = 1

# for the dataframe batch2_3_Q_0_37 drop everything after row 36
batch2_3_Q_0_37 = batch2_3_Q_0_37.drop(batch2_3_Q_0_37.index[37:])

# append batch2_3_Q_0_37 to batch2_3
batch2_3 = batch2_3.append(batch2_3_Q_0_37, ignore_index=True)

# create a list of all the dataframes
df_list = [batch1_df, batch2_3, batch4_5, batch6_7, batch8_9, batch10]

# append all the dataframes together
df_master = pd.DataFrame()

for df in df_list:
    df_master = df_master.append(df, ignore_index=True)

# write the master dataframe to a csv file
df_master.to_csv('output/master.csv', index=False)


C:\Users\cayde\AppData\Local\Temp\ipykernel_19448\2815882531.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  batch1_df = batch1_df.append(temp_df, ignore_index=True)
C:\Users\cayde\AppData\Local\Temp\ipykernel_19448\2815882531.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  batch2_3 = batch2_3.append(batch2_3_Q_0_37, ignore_index=True)
C:\Users\cayde\AppData\Local\Temp\ipykernel_19448\2815882531.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_master = df_master.append(df, ignore_index=True)


In [24]:
# the column 'agents_score' is a long string that starts will CORRECT or INCORRECT count the occurances of CORRECT and INCORRECT and print the results
correct_count = (df_master['agents_score'].str.startswith('CORRECT')).sum()
incorrect_count = (df_master['agents_score'].str.startswith('INCORRECT')).sum()

print('Agents results:')
print(f'agents CORRECT count: {correct_count} | agents INCORRECT count: {incorrect_count} ')

failed_to_answer_wrong_count = ((df_master['agent_answer'] == 'failed to answer') & (df_master['agents_score'].str.startswith('INCORRECT'))).sum()

print(f'system failure count: {failed_to_answer_wrong_count}\n')

# do the same for the column 'gpt-4_score'
correct_count = (df_master['gpt-4_score'].str.startswith('CORRECT')).sum()
incorrect_count = (df_master['gpt-4_score'].str.startswith('INCORRECT')).sum()

print('GPT-4 results:')
print(f'gpt-4 CORRECT count: {correct_count} | gpt-4 INCORRECT count: {incorrect_count}\n')

correct_agent_incorrect_gpt = ((df_master['agents_score'].str.startswith('CORRECT')) & (df_master['gpt-4_score'].str.startswith('INCORRECT'))).sum()
incorrect_agent_correct_gpt = ((df_master['agents_score'].str.startswith('INCORRECT')) & (df_master['gpt-4_score'].str.startswith('CORRECT'))).sum()

print('compare stats:')
print(f'Agent got correct, GPT-4 incorrect: {correct_agent_incorrect_gpt}')
print(f'Agent got incorrect(including "system failure"), GPT-4 correct: {incorrect_agent_correct_gpt}')

wrong_answer_correct_gpt_count = ((df_master['agent_answer'] != 'failed to answer') & (df_master['agents_score'].str.startswith('INCORRECT')) & (df_master['gpt-4_score'].str.startswith('CORRECT'))).sum()

print(f'Agent got incorrect(excluding "system failure"), GPT-4 correct: {wrong_answer_correct_gpt_count}')

Agents results:
agents CORRECT count: 440 | agents INCORRECT count: 760 
system failure count: 459

GPT-4 results:
gpt-4 CORRECT count: 572 | gpt-4 INCORRECT count: 628

compare stats:
Agent got correct, GPT-4 incorrect: 133
Agent got incorrect(including "system failure"), GPT-4 correct: 265
Agent got incorrect(excluding "system failure"), GPT-4 correct: 91
